In [1]:
!curl -X GET http://172.18.0.99:9092

curl: (52) Empty reply from server


In [2]:
!/opt/kafka/bin/kafka-topics.sh --list --bootstrap-server 172.18.0.99:9092

__consumer_offsets
phongdinhcs-test-topic


In [3]:
!/opt/kafka/bin/kafka-topics.sh --create --topic phongdinhcs-test-topic --bootstrap-server 172.18.0.99:9092 --partitions 1 --replication-factor 1

Error while executing topic command : Topic 'phongdinhcs-test-topic' already exists.
[2024-08-06 23:24:37,581] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'phongdinhcs-test-topic' already exists.
 (org.apache.kafka.tools.TopicCommand)


In [1]:
!/opt/kafka/bin/kafka-topics.sh --list --bootstrap-server 172.18.0.99:9092

__consumer_offsets
phongdinhcs-test-topic


In [2]:
!/opt/kafka/bin/kafka-consumer-groups.sh --bootstrap-server 172.18.0.99:9092 --describe --group my-group


Consumer group 'my-group' has no active members.

GROUP           TOPIC                  PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID     HOST            CLIENT-ID
my-group        phongdinhcs-test-topic 0          1               20              19              -               -               -

In [6]:
from kafka import KafkaProducer

# Create a Kafka producer
producer = KafkaProducer(
    bootstrap_servers='172.18.0.99:9092',
    value_serializer=lambda v: str(v).encode('utf-8')  # Convert messages to bytes
)

# Send a message to the topic
producer.send('phongdinhcs-test-topic', value='Hello, Kafka!')

# Wait for all messages to be sent
producer.flush()

print("Message sent to topic 'phongdinhcs-test-topic'")


Message sent to topic 'phongdinhcs-test-topic'


In [7]:
from kafka import KafkaConsumer

# Create a Kafka consumer
consumer = KafkaConsumer(
    'phongdinhcs-test-topic',
    bootstrap_servers='172.18.0.99:9092',
    group_id='my-group',
    value_deserializer=lambda x: x.decode('utf-8')  # Decode bytes to string
)

# Consume messages from the topic
for message in consumer:
    print(f"Received message: {message.value}")
    break  # Remove this break to continuously consume messages


Received message: Hello, Kafka!


In [28]:
# Show tables in the current database

spark.sql("SHOW TABLES").show()

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|  default|teststocksymbols|      false|
+---------+----------------+-----------+



In [3]:
# Describe all information in metastore
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Metastore Metadata") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Show all databases
databases = spark.sql("SHOW DATABASES").collect()

# Loop through each database
for db in databases:
    db_name = db[0]  # Access the first column (database name)
    spark.sql(f"USE {db_name}")
    tables = spark.sql("SHOW TABLES").collect()
    print(f"\nDatabase: {db_name}\n" + "-" * 40)
    
    # Loop through each table
    for table in tables:
        table_name = table[1]  # Access the second column (table name)
        
        # Count the number of rows in the table
        row_count = spark.sql(f"SELECT COUNT(*) AS count FROM {table_name}").collect()[0][0]
        
        print(f"Table: {table_name} (Rows: {row_count})")
        print("-" * 40)
        
        # Describe each table
        description = spark.sql(f"DESCRIBE TABLE {table_name}")
        schema = description.schema
        print(f"{'Column Name':<15} | {'Data Type':<10} | {'Nullable':<10} | {'Comment':<15}")
        print("-" * 50)
        
        for row in description.collect():
            col_name = row[0]         # Column name
            data_type = row[1]        # Data type
            nullable = row[2]         # Exact nullable status
            
            # Handle optional comment field if present
            if len(row) > 3:
                comment = row[3]      # Column comment (if available)
            else:
                comment = "N/A"       # Default if no comment
            
            # Print with better formatting
            print(f"{col_name:<15} | {data_type:<10} | {nullable:<10} | {comment:<15}")
        print("-" * 50)


24/08/11 03:36:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.

Database: default
----------------------------------------
Table: teststocksymbols (Rows: 6)
----------------------------------------
Column Name     | Data Type  | Nullable   | Comment        
--------------------------------------------------
symbol          | string     |            | N/A            
name            | string     |            | N/A            
price           | double     |            | N/A            
volume          | int        |            | N/A            
                |            |            | N/A            
# Partitioning  |            |            | N/A            
Not partitioned |            |            | N/A            
--------------------------------------------------


In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session with Delta configurations
spark = SparkSession.builder \
    .appName("TableDetailMetadata") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Function to get table details
def get_table_details(database_name, table_name):
    try:
        details = spark.sql(f"DESCRIBE DETAIL {database_name}.{table_name}").collect()
        if details:
            detail_info = details[0].asDict()
            return detail_info
        else:
            return {"error": "Details not found"}
    except Exception as e:
        return {"error": str(e)}

# Get all databases
databases = spark.sql("SHOW DATABASES").collect()

# Dictionary to hold metadata for all tables
all_tables_metadata = {}

for db in databases:
    db_name = db.namespace
    spark.sql(f"USE {db_name}")

    # Get all tables in the database
    tables = spark.sql("SHOW TABLES").collect()
    table_metadata = {}

    for table in tables:
        table_name = table.tableName
        print(f"Fetching details for table: {table_name} in database: {db_name}")
        
        # Get table details
        details = get_table_details(db_name, table_name)
        table_metadata[table_name] = details

    all_tables_metadata[db_name] = table_metadata

# Display the metadata
for db_name, tables in all_tables_metadata.items():
    print(f"\nDatabase: {db_name}")
    for table_name, details in tables.items():
        print(f"\nTable: {table_name}")
        for key, value in details.items():
            print(f"{key}: {value}")


Fetching details for table: teststocksymbols in database: default

Database: default

Table: teststocksymbols
format: delta
id: 806ed304-8330-4e8e-87be-997af91087fb
name: default.teststocksymbols
description: None
location: hdfs://namenode:8020/testlakehouse/TestStockSymbols
createdAt: 2024-07-19 00:40:02.374000
lastModified: 2024-08-06 23:23:24.342000
partitionColumns: []
numFiles: 6
sizeInBytes: 7556
properties: {}
minReaderVersion: 1
minWriterVersion: 2
tableFeatures: ['appendOnly', 'invariants']
